In [1]:
# Import the python packages you will need 
import pandas 
import numpy as np
import netCDF4 as nc
import xarray as xr
import xesmf
import datetime as dt
from scipy import stats
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
# NEP Bering Grid 
bering_nep_grid_file = '/work/role.medgrp/NEP/plotting/Figure_19_20/nep_bering_grid.nc'
grd_fid = nc.Dataset(bering_nep_grid_file)

# extracting lat/lon from geolat/geolon
nep_lat = grd_fid.variables['geolat'][:]
nep_lon = grd_fid.variables['geolon'][:]

#### Interpolation for Eastern and Northern Bering Sea for Kearney spatial plotting

In [3]:
# daily NEP bering sea bottom temp
nep_fil = '/work/role.medgrp/NEP/plotting/Figure_19_20/nep_tob_bering_1993-2024_daily_ts.nc'
nep_ds = xr.open_dataset(nep_fil)
nep_ds = nep_ds.assign_coords(lon = (('yh', 'xh'), nep_lon))
nep_ds = nep_ds.assign_coords(lat = (('yh', 'xh'), nep_lat))

## daily bottom temp
#trawl_fil = './ebs_nbs_temperature_full_area.csv'
trawl_fil = '/work/role.medgrp/NEP/TOOLS/coldpool/data/ebs_nbs_temperature_full_area.csv'
df = pandas.read_csv(trawl_fil)

# drop rows that go back for forward in time further than the temporal extent of the NEP hindcast
df = df[df.year >= 1993].reset_index(drop=True)
#df = df[df.year <= 2019].reset_index(drop=True)

# only measurements with haul type 3
df = df[df.haul_type == 3].reset_index(drop=True)

# generate copy of trawl dataframe with added empty column for interpolated NEP temp
df2 = df.assign(nep_tob=np.nan*np.zeros(len(df)))

trawl_locs = xr.Dataset()
start_time = time.time()

# run interpolation step using xesmf
for row in df[:200].itertuples():
    index=row[0]
    year = row[-1]
    
    if index%200 == 0:
        print(index,"--- %s seconds ---" % (time.time() - start_time)) # Tracking progress
    
    # get lat lon values for regridding
    trawl_locs['lon'] = xr.DataArray(data=[df['longitude'][index]], dims=('trawl_sample'))
    trawl_locs['lat'] = xr.DataArray(data=[df['latitude'][index]], dims=('trawl_sample'))
    
    # create unique regridder for trawl location
    nep_regridder = xesmf.Regridder(nep_ds, trawl_locs, 'bilinear', locstream_out=True)

    # regrid NEP and GLORYS bottom temperature value and interpolate time - Alaska Daylight Time: UTC-8
    df2['nep_tob'][index] = nep_regridder(nep_ds['tob'].interp(time=(dt.datetime.strptime(df['start_time'][index], '%Y-%m-%d %H:%M:%S')+dt.timedelta(hours=8))))
 
new_csv = 'ebs_nbs_temperature_full_area_nep.csv'
df2.to_csv(new_csv)

0 --- 0.0026450157165527344 seconds ---
For 0: interped to <xarray.DataArray 'tob' (yh: 150, xh: 150)> Size: 180kB
array([[ 1.12579787,  1.12564047,  1.12592632, ...,         nan,
                nan,         nan],
       [ 1.1263053 ,  1.12567319,  1.12576464, ...,         nan,
                nan,         nan],
       [ 1.1270263 ,  1.12586257,  1.12566799, ...,         nan,
                nan,         nan],
       ...,
       [ 1.23680528,  1.23720357,  1.23733112, ..., -1.74617149,
        -1.74512367, -1.742714  ],
       [ 1.23618174,  1.23680313,  1.23715988, ..., -1.75171502,
        -1.75028123, -1.74765912],
       [ 1.23557025,  1.23617995,  1.23687679, ..., -1.7548233 ,
        -1.75357546, -1.75111812]])
Coordinates:
  * xh       (xh) float64 1kB 233.6 233.6 233.7 233.8 ... 244.5 244.6 244.6
  * yh       (yh) float64 1kB 51.52 51.57 51.62 51.67 ... 57.56 57.59 57.62
    lon      (yh, xh) float32 90kB 195.4 195.5 195.6 195.7 ... 185.3 185.4 185.5
    lat      (yh, xh) floa


KeyboardInterrupt



#### Interpolation for Index Bering Sea for Rohan CPA index processing and analysis

In [4]:
## daily bottom temp
trawl_fil = '/work/role.medgrp/NEP/TOOLS/coldpool/data/index_hauls_temperature_data.csv'
df = pandas.read_csv(trawl_fil)

# drop rows that go back for forward in time further than the temporal extent of the NEP hindcast
df = df[df.year >= 1993].reset_index(drop=True)
#df = df[df.year <= 2019].reset_index(drop=True)

# append fake 2020 time values for nep regridding
df_2020 = df[df.year==2019].reset_index(drop=True)
df_2020.year = 2020
df_2020.gear_temperature = np.nan
df_2020.surface_temperature  = np.nan
df_2020.cruise = np.nan
for row in df_2020.itertuples():
    index=row[0]
    time_str = df_2020.start_time[index].split('2019')[1] 
    df_2020.start_time[index] = ('2020' + time_str)

df = pandas.concat([df, df_2020]).reset_index(drop=True)

nep_fil = '/work/role.medgrp/NEP/plotting/Figure_19_20/nep_tob_bering_1993-2024_daily_ts.nc'
nep_ds = xr.open_dataset(nep_fil)
nep_ds = nep_ds.assign_coords(lon = (('yh', 'xh'), nep_lon))
nep_ds = nep_ds.assign_coords(lat = (('yh', 'xh'), nep_lat))

# generate copy of trawl dataframe with added empty column for interpolated NEP temp
df2 = df.assign(nep_tob=np.nan*np.zeros(len(df)))

trawl_locs = xr.Dataset()
start_time = time.time()

# run interpolation step using xesmf
for row in df.itertuples():
    index=row[0]
    year = row[-1]
    
    #print(dt.datetime.strptime(df['start_time'][index], '%Y-%m-%d %H:%M:%S'))
    if index%200 == 0:
        print(index,"--- %s seconds ---" % (time.time() - start_time)) # Tracking progress
    
    # get lat lon values for regridding
    trawl_locs['lon'] = xr.DataArray(data=[df['longitude'][index]], dims=('trawl_sample'))
    trawl_locs['lat'] = xr.DataArray(data=[df['latitude'][index]], dims=('trawl_sample'))
    
    # create unique regridder for trawl location
    nep_regridder = xesmf.Regridder(nep_ds, trawl_locs, 'bilinear', locstream_out=True)
    
    # regrid NEP and GLORYS bottom temperature value and interpolate time - Alaska Daylight Time: UTC-8
    df2['nep_tob'][index] = nep_regridder(nep_ds['tob'].interp(time=(dt.datetime.strptime(df['start_time'][index], '%Y-%m-%d %H:%M:%S')+dt.timedelta(hours=8))))
        
new_csv = 'index_hauls_temperature_data_nep.csv'
df2.to_csv(new_csv)

0 --- 0.0007302761077880859 seconds ---
200 --- 282.2391595840454 seconds ---
400 --- 563.3549175262451 seconds ---
600 --- 844.6057896614075 seconds ---
800 --- 1128.1893458366394 seconds ---
1000 --- 1412.3896386623383 seconds ---
1200 --- 1695.223258972168 seconds ---
1400 --- 1980.0863256454468 seconds ---
1600 --- 2262.1435692310333 seconds ---
1800 --- 2542.373544692993 seconds ---
2000 --- 2822.567291021347 seconds ---
2200 --- 3102.462597846985 seconds ---
2400 --- 3382.3165674209595 seconds ---
2600 --- 3661.108823776245 seconds ---
2800 --- 3939.9473853111267 seconds ---
3000 --- 4218.360578298569 seconds ---
3200 --- 4496.50536775589 seconds ---
3400 --- 4774.19514799118 seconds ---
3600 --- 5052.7199602127075 seconds ---
3800 --- 5333.048392772675 seconds ---
4000 --- 5611.6676251888275 seconds ---
4200 --- 5889.750591754913 seconds ---
4400 --- 6167.881146669388 seconds ---
4600 --- 6448.606292247772 seconds ---
4800 --- 6727.951157331467 seconds ---
5000 --- 7010.25025177